In [30]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [31]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # input image channel, 6 output channels, 3x3 square convolution
        
        self.conv1 = nn.Conv2d(1, 6, 3)
        self.conv2 = nn.Conv2d(6, 16, 3)
        # an affite operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 6 * 6, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        
    def forward(self, x):
        # max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # if the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))        
        x = self.fc3(x)
        return x
    
    def num_flat_features(self, x):
        size = x.size()[1:] # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

net = Net()
net

Net(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

In [32]:
params = list(net.parameters())
print(len(params))

10


In [33]:
print(params[0].size()) # conv1's weight

torch.Size([6, 1, 3, 3])


In [34]:
params[0]

Parameter containing:
tensor([[[[ 0.2008, -0.1228, -0.3268],
          [ 0.2116, -0.3006, -0.1239],
          [ 0.2736, -0.1326,  0.2562]]],


        [[[ 0.1864, -0.2100, -0.1847],
          [-0.0988,  0.1502,  0.2358],
          [-0.1600,  0.2744, -0.2091]]],


        [[[ 0.2559,  0.1953,  0.1318],
          [-0.3113, -0.0296,  0.1356],
          [-0.0162,  0.1812,  0.0790]]],


        [[[ 0.1122,  0.2797,  0.1680],
          [ 0.2318, -0.0917,  0.0745],
          [-0.2207,  0.1567,  0.1623]]],


        [[[ 0.3075, -0.1156, -0.2218],
          [ 0.0147, -0.1723, -0.2513],
          [ 0.0752,  0.1129, -0.1037]]],


        [[[ 0.0145, -0.0865, -0.0905],
          [-0.1086, -0.3189, -0.0280],
          [ 0.0959,  0.2828,  0.2567]]]], requires_grad=True)

In [35]:
inp = torch.randn(1, 1, 32, 32)
out = net(inp)
out

tensor([[ 0.0035,  0.0894,  0.0250,  0.1150,  0.0389, -0.0109,  0.0149, -0.0449,
          0.1235,  0.0888]], grad_fn=<AddmmBackward>)

Zero the gradient buffers of all parameters and backprops with random gradients

In [36]:
net.zero_grad()
out.backward(torch.randn(1, 10))

In [37]:
output = net(inp)
target = torch.randn(10)
target = target.view(1, -1)
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)

tensor(1.3670, grad_fn=<MseLossBackward>)


When we call `loss.backward()` the whole graph is differentiated w.r.t. the loss and all Tenors in the graph that has `requires_grad=True` will have their `.grad` Tensor accumulated with the gradient.

In [38]:
print(loss.grad_fn) #MSELoss
print(loss.grad_fn.next_functions[0][0]) # Linear
print(loss.grad_fn.next_functions[0][0].next_functions[0][0]) # ReLU

### Backprop

In [39]:
net.zero_grad()

print("conv1.bias.grad before backward")
print(net.conv1.bias.grad)

loss.backward()

print("conv1.bias.grad after backward")
print(net.conv1.bias.grad)

conv1.bias.grad before backward
tensor([0., 0., 0., 0., 0., 0.])
conv1.bias.grad after backward
tensor([-0.0066,  0.0133,  0.0093,  0.0094, -0.0009, -0.0011])


#### Updating the weights

`weight = weight - learning_rate * gradient`

In [40]:
lr = 0.01
for f in net.parameters():
    f.data.sub_(f.grad.data * lr)

In [41]:
import torch.optim as optim

optimizer = optim.SGD(net.parameters(), lr=lr)

optimizer.zero_grad()
output = net(inp)
loss = criterion(output, target)
loss.backward()
optimizer.step()